James Gardner 2022

In [ ]:
# import packages
import os, sys
import numpy as np
import matplotlib.pyplot as plt  
%matplotlib inline

from pprint import pprint

from pykat import finesse
from pykat.commands import *
pykat.init_pykat_plotting(dpi=90)

In [ ]:
# base of configurations, base model to then add components to
# reference: http://www.gwoptics.org/finesse/examples/aligo_sensitivity.php
# namely, doesn't contain connecting component in SRC between BS and SRM
# later functions either add a space, or space-nle-space there
basekat = finesse.kat()
basecode = """
# spaces
const lx 1.6
const ly 1.55
const Lx 3994.469
const Ly 3994.469
const lprc 53
# mirrors

# power recycling
s sBStoPRM $lprc nPRM nBSi

# central beam splitter
bs BS 0.5 0.5 0 45 nBSi nBSr nBSt nBSo
s sBStoYarm $ly nBSr nY0
s sBStoXarm $lx nBSt nX0

# Y arm (perpendicular)
s sY $Ly nY1 nY2

# X arm (parallel)
s sX $Lx nX1 nX2

# signal recycling
# nBSo to nSRM left disconnected for now
# is connected differently in kat1 and kat2
s sSRMtohdBS 0 nDark nhdBSi # null length space
"""
basekat.parse(basecode)

# print((basekat.components, basekat.detectors, basekat.commands))

In [ ]:
# use this function to check if powers are correct
def check_tuning(kat):
    """checking IFO tuning by seeing if circulating power in arms is close to 800kW
    """
    check_tuning_kat = deepcopy(kat)
    check_tuning_kat.verbose = False
    check_tuning_code="""
    pd initial nLaser*
    pd circPRC nPRM
    pd circX nX1
    pd circY nY1

    pd BSi nBSi*
    pd BSr nBSr
    pd BSt nBSt
    pd BSo nBSo

    noxaxis
    """
    check_tuning_kat.parse(check_tuning_code)
    check_tuning_out = check_tuning_kat.run()
    
    input_power = check_tuning_out["initial"].mean()
    circulating_arm_cavity_power = check_tuning_out["circX"].mean()
    print(f"Input power, Pin: {input_power:.3g} W\nPower incident on beamsplitter: {check_tuning_out['BSi'].mean()*1e-3:.3g} kW\nPower transmitted from beamsplitter towards X-arm: {check_tuning_out['BSt'].mean()*1e-3:.3g} kW\nCirculating arm cavity power, Pcirc: {circulating_arm_cavity_power*1e-6:.3g} MW")

def create_kat1(lsrc=56, pin=125, titm=0.014, label="No nle", t_srm=0.046, loss_srm=1000e-6, loss_arm=100e-6, t_prm=0.03):
    """returns basekat with a space as the connecting component
    configuration 1: connecting nBSo and nSRM with a space
    lsrc in m
    """
    kat1 = deepcopy(basekat)
    kat1code = f"""
    const Loss_itm 0
    const Mtm 200 # kg
    # --------------
    const Pin {pin}
    const Titm {titm} # aLIGO uses 0.014, Voyager uses 0.002, Korobko+2019 uses 0.07, I use 0.197
    l laser $Pin 0 nLaser
    
    const Tprm {t_prm}
    const Loss_prm 0
    m1 PRM $Tprm $Loss_prm 90 nLaser nPRM # tuning = 90° towards nLaser
    
    m1 mYitm $Titm $Loss_itm 0 nY0 nY1
    m1 mXitm $Titm $Loss_itm 90 nX0 nX1 # tuning = 90° towards nX0
    attr mYitm mass $Mtm
    attr mXitm mass $Mtm
   
    const Tetm 0
    const Loss_etm {loss_arm}
    m1 mYetm $Tetm $Loss_etm 0 nY2 nY3
    attr mYetm mass $Mtm
    m1 mXetm $Tetm $Loss_etm 90 nX2 nX3 # tuning = 90° towards nX2
    attr mXetm mass $Mtm 
    # --------------    
    const lsrc {lsrc}
    s sBStoSRM $lsrc nBSo nSRM
    # --------------    
    const Tsrm {t_srm}
    const Loss_srm {loss_srm}
    m1 SRM $Tsrm $Loss_srm 90 nSRM nDark
    """
    kat1.parse(kat1code)
    
    print('--------------------')
    print(f'{label}.\nITM transmission, Titm: {titm}')
    
    return kat1

In [ ]:
# Adhikari+2020 Voyager
settings = dict(pin=152, lsrc=56, titm=2e-3, t_srm=4.6e-2, t_prm=4.9e-2, loss_srm=0, loss_arm=0)
check_tuning(create_kat1(label="Adhikari+2020", **settings))
pprint(settings)
# 152 W input, 3100 W (3.1 kW) in PRC, 3 MW in arms
# ifo = IFO(lambda0=2e-6, L_arm=4e3, L_SRC=56, P_circ=3e6, T_ITM=0.002, T_SRM=0.046, M=200, ws=None, gbR=None, gcR=0)

In [ ]:
# want to achieve 5.3 kW on the BS
settings = dict(pin=40.35, lsrc=53, t_srm=0.325, loss_srm=35e-6, loss_arm=0)
check_tuning(create_kat1(label="Adya+2020", **settings))
pprint(settings)

In [ ]:
# isn't lsrc in nIS model, since extra length between bs and itm needs adding in.
lsrc0 = 1124 - (1.6 + 1.55)/2
check_tuning(create_kat1(label="Voyager", lsrc=56, pin=1305, titm=0.002))
check_tuning(create_kat1(label="aLIGO", lsrc=56, pin=601, titm=0.014))
check_tuning(create_kat1(label="Korobko+2019", lsrc=56, pin=1154, titm=0.07))
check_tuning(create_kat1(label="Table 1", lsrc=56, pin=2815, titm=0.197))

Is the carrier wavelength correct?